In [1]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Combine speed restriction data for 2025
jan_speed_restrictions = pd.read_csv('2025-01_Speed_Restrictions_By_Day.csv')
feb_speed_restrictions = pd.read_csv('2025-02_Speed_Restrictions_By_Day.csv')

combined_speed_restrictions = pd.concat([jan_speed_restrictions, feb_speed_restrictions])

# fix dates
combined_speed_restrictions['Calendar_Date'] = pd.to_datetime(combined_speed_restrictions['Calendar_Date'])\

# Group to get restriction count
grouped_speed_restrictions = combined_speed_restrictions.groupby(by="Branch", as_index=False)["Restriction_Days_Active_On_Calendar_Day"].sum()
grouped_speed_restrictions

In [ ]:
# bar plot (frequency of speed restictions by line)
colors = ['blue', 'green', 'green', 'green', 'darkorange', 'darkred', 'darkred', 'darkred']
plt.bar(grouped_speed_restrictions['Branch'], grouped_speed_restrictions['Restriction_Days_Active_On_Calendar_Day'], color = colors)
plt.ylabel("Number of Days with Speed Restriction")
plt.xlabel("Line")
plt.xticks(rotation=50)
plt.title("Reported Speed Restrictions by Line in 2025")

In [ ]:
ridership_data = pd.read_csv('MBTA_Monthly_Ridership_By_Mode_and_Line.csv')

# fix dates (remove times)
ridership_data['month_of_service'] = pd.to_datetime(ridership_data['month_of_service'])

# exclude missing ridership data
filtered_ridership = ridership_data[pd.notnull(ridership_data.ridership_total)]
filtered_ridership = filtered_ridership[filtered_ridership.daytype == "Total"]

# filter by most popular lines
pop_lines = ['Bus', 'Commuter Rail', 'Green Line', 'Orange Line', 'Red Line', 'Silver Line', 'Blue Line']
filtered_ridership = filtered_ridership[filtered_ridership['route_or_line'].isin(pop_lines)]

filtered_ridership

In [ ]:
# Ridership Line Plot
# Create a selection widget for filtering
line_selection = alt.selection_multi(fields=['route_or_line'], bind='legend')

# Changing colors to match line
color_scale = alt.Scale(domain=pop_lines,
                        range=['goldenrod', 'purple', 'green', 'darkorange', 'red', 'silver', 'blue']) 

alt.Chart(filtered_ridership).mark_line().encode(
    alt.X('month_of_service:T', title="Month of Service", axis=alt.Axis(format="%b %Y")),
    alt.Y('ridership_total:Q', title="Total Ridership"),
    color=alt.Color('route_or_line:N', scale=color_scale, legend=alt.Legend(title="Route or Line")),
    tooltip=['month_of_service:T', 'ridership_total:Q', 'route_or_line:N'],
    opacity=alt.condition(line_selection, alt.value(1), alt.value(0.1))
).properties(
    width = 400, height = 400,
    title="Ridership Over Time"
).add_selection(
    line_selection
)